In [1]:
# Step 1: Get Data from RxNorm API
import requests
import json

def get_rxnorm_data(drug_name):
    """
    Fetches RxNorm data for a given drug name.
    
    Args:
        drug_name (str): The name of the drug to search for.
    
    Returns:
        dict: The API response as a JSON object or None if the request fails.
    """
    url = f"https://rxnav.nlm.nih.gov/REST/drugs.json?name={drug_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Unable to fetch data for {drug_name}. Status code: {response.status_code}")
        return None

def extract_rxcui(rxnorm_data):
    """
    Extracts the RxCUI (RxNorm Concept Unique Identifier) from the API response.
    
    Args:
        rxnorm_data (dict): The JSON response from RxNorm API.
    
    Returns:
        str: The RxCUI of the drug or None if not found.
    """
    try:
        return rxnorm_data['drugGroup']['conceptGroup'][0]['conceptProperties'][0]['rxcui']
    except (KeyError, IndexError, TypeError):
        print("RxCUI not found in the response.")
        return None

def save_to_json(data, filename="rxnorm_data.json"):
    """
    Saves the API response to a JSON file.
    
    Args:
        data (dict): The JSON data to save.
        filename (str): The name of the output JSON file.
    """
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4)
    print(f"Data successfully saved to {filename}")


In [2]:
# step 2 save file

# Save RxNorm data

# Call RxNorm  API
drug = "Ibuprofen"
rx_data = get_rxnorm_data(drug)
save_to_json(rx_data, "ibuprofen_rxnorm.json")

Data successfully saved to ibuprofen_rxnorm.json


In [3]:
# Step 3 : Get Data from openFDA API
import requests
import json

def get_openfda_data(drug_name):
    """
    Fetches OpenFDA drug label data for a given drug name.
    
    Args:
        drug_name (str): The name of the drug to search for.
    
    Returns:
        dict: The API response as a JSON object or None if the request fails.
    """
    url = f"https://api.fda.gov/drug/label.json?search=generic_name:{drug_name}&limit=1"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Unable to fetch data for {drug_name}. Status code: {response.status_code}")
        return None

def extract_drug_info(openfda_data):
    """
    Extracts relevant drug information from the API response.
    
    Args:
        openfda_data (dict): The JSON response from OpenFDA API.
    
    Returns:
        dict: Extracted drug details (brand name, manufacturer, purpose) or None if not found.
    """
    try:
        drug_info = openfda_data['results'][0]  # Access first result
        extracted_data = {
            "brand_name": drug_info.get("openfda", {}).get("brand_name", ["Unknown"])[0],
            "manufacturer": drug_info.get("openfda", {}).get("manufacturer_name", ["Unknown"])[0],
            "purpose": drug_info.get("purpose", ["No purpose information available"])[0]
        }
        return extracted_data
    except (KeyError, IndexError, TypeError):
        print("Drug information not found in the response.")
        return None

def save_to_json(data, filename="openfda_data.json"):
    """
    Saves the API response to a JSON file.
    
    Args:
        data (dict): The JSON data to save.
        filename (str): The name of the output JSON file.
    """
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4)
    print(f"Data successfully saved to {filename}")

In [4]:
import requests

def get_openfda_data(drug_name):
    url = f'https://api.fda.gov/drug/label.json?search=openfda.brand_name:"{drug_name}"'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Unable to fetch data for {drug_name}. Status code: {response.status_code}")
        return None

# Call API
openfda_data = get_openfda_data("IBUPROFEN")  # uppercase works better


In [5]:

# Save OpenFDA data
# Call OpenFDA API
openfda_data = get_openfda_data("Ibuprofen")

if openfda_data is not None:
    save_to_json(openfda_data, "ibuprofen_openfda.json")
else:
    print("OpenFDA API returned no data")

Data successfully saved to ibuprofen_openfda.json


In [6]:
import os
os.getcwd()


'D:\\1 Univesrity work\\Lect\\Data  management\\Drug-API-Integrator\\Data and Project'

In [7]:
# Step 3: Merge RxNorm and openFDA Data
def integrate_data(rxnorm_data, openfda_data):
    # Check if openfda_data is None or empty
    if openfda_data is None or not openfda_data.get("results"):
        openfda_warnings = "Not Available"
        openfda_purpose = "Not Available"
    else:
        openfda_warnings = openfda_data["results"][0].get("warnings", "Not Available")
        openfda_purpose = openfda_data["results"][0].get("purpose", "Not Available")
    
    integrated_data = {
        "drug_name": rxnorm_data["drugGroup"]["name"],
        "rxcui": extract_rxcui(rxnorm_data),
        "openfda_warnings": openfda_warnings,
        "openfda_purpose": openfda_purpose
    }
    
    return integrated_data


In [8]:
# Create Database 
import sqlite3
conn = sqlite3.connect("medicine.db")  # database file
cursor = conn.cursor()




In [9]:
# Creating table drug_data
cursor.execute("""
CREATE TABLE IF NOT EXISTS drug_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    drug_name TEXT,
    rxcui TEXT,
    openfda_warnings TEXT,
    openfda_purpose TEXT
)
""")
conn.commit()


In [10]:
# integrated data (from RxNorm + OpenFDA)
integrated_data = {
    "drug_name": "Ibuprofen",
    "rxcui": "12345",  # replace with actual RxCUI from extract_rxcui()
    "openfda_warnings": "Use with caution",
    "openfda_purpose": "Pain relief"
}


cursor.execute("""
INSERT INTO drug_data (drug_name, rxcui, openfda_warnings, openfda_purpose)
VALUES (?, ?, ?, ?)
""", (integrated_data["drug_name"], integrated_data["rxcui"], 
      integrated_data["openfda_warnings"], integrated_data["openfda_purpose"]))

conn.commit()

In [11]:
# Watching the result after the integrated

conn = sqlite3.connect("medicine.db")
conn.row_factory = sqlite3.Row  # returns dict-like rows
cursor = conn.cursor()

cursor.execute("SELECT * FROM drug_data")
rows = cursor.fetchall()

for row in rows:
    print(dict(row))  # convert Row to dictionary for nice display





{'id': 1, 'drug_name': 'Ibuprofen', 'rxcui': '12345', 'openfda_warnings': 'Use with caution', 'openfda_purpose': 'Pain relief'}
{'id': 3, 'drug_name': 'Warfarin', 'rxcui': '36870', 'openfda_warnings': 'Take with food', 'openfda_purpose': 'Allergy relief'}
{'id': 4, 'drug_name': 'Simvastatin', 'rxcui': '47565', 'openfda_warnings': 'Take with food', 'openfda_purpose': 'Blood thinning'}
{'id': 5, 'drug_name': 'Aspirin', 'rxcui': '10611', 'openfda_warnings': 'Consult doctor if pregnant', 'openfda_purpose': 'Pain relief'}
{'id': 6, 'drug_name': 'Tamsulosin', 'rxcui': '23867', 'openfda_warnings': 'May cause drowsiness', 'openfda_purpose': 'Pain relief'}
{'id': 7, 'drug_name': 'Prednisone', 'rxcui': '97603', 'openfda_warnings': 'Use with caution', 'openfda_purpose': 'Lower blood pressure'}
{'id': 8, 'drug_name': 'Amoxicillin', 'rxcui': '52936', 'openfda_warnings': 'Consult doctor if pregnant', 'openfda_purpose': 'Lower blood pressure'}
{'id': 9, 'drug_name': 'Escitalopram', 'rxcui': '85693', 

In [12]:
import random
import sqlite3

# Connecting to SQLite database
conn = sqlite3.connect("medicine.db")
cursor = conn.cursor()

# Sample data lists
drug_names = [
    "Aspirin", "Ibuprofen", "Acetaminophen", "Metformin", "Atorvastatin", "Simvastatin",
    "Losartan", "Lisinopril", "Omeprazole", "Pantoprazole", "Amoxicillin", "Azithromycin",
    "Ciprofloxacin", "Doxycycline", "Prednisone", "Albuterol", "Warfarin", "Clopidogrel",
    "Insulin", "Hydrochlorothiazide", "Levothyroxine", "Metoprolol", "Gabapentin", "Hydrocodone",
    "Sertraline", "Furosemide", "Fluoxetine", "Escitalopram", "Tamsulosin", "Duloxetine"
]

warnings_list = [
    "Not Recommended for children", "Take with food", "Avoid alcohol", "Consult doctor if pregnant",
    "May cause drowsiness", "Use with caution", "Monitor blood pressure"
]

purposes_list = [
    "Pain relief", "Reduce inflammation", "Treat bacterial infection", "Lower blood pressure",
    "Manage diabetes", "Allergy relief", "Cholesterol control", "Blood thinning"
]

# Generate 200 random entries
drug_entries = []
for _ in range(200):
    drug_entries.append((
        random.choice(drug_names),            # drug_name
        str(random.randint(10000, 99999)),    # rxcui as random number
        random.choice(warnings_list),         # openfda_warnings
        random.choice(purposes_list)          # openfda_purpose
    ))

# Insert multiple rows into SQL table
cursor.executemany("""
INSERT INTO drug_data (drug_name, rxcui, openfda_warnings, openfda_purpose)
VALUES (?, ?, ?, ?)
""", drug_entries)
conn.commit()
print("✅ Successfully inserted 200 random drugs into drug_data")

# watching and display to verify
import pandas as pd
df = pd.read_sql_query("SELECT * FROM drug_data", conn)
print(df.head())



✅ Successfully inserted 200 random drugs into drug_data
   id    drug_name  rxcui            openfda_warnings openfda_purpose
0   1    Ibuprofen  12345            Use with caution     Pain relief
1   3     Warfarin  36870              Take with food  Allergy relief
2   4  Simvastatin  47565              Take with food  Blood thinning
3   5      Aspirin  10611  Consult doctor if pregnant     Pain relief
4   6   Tamsulosin  23867        May cause drowsiness     Pain relief


In [13]:
# watching data with a filter
df = pd.read_sql_query("""
SELECT drug_name, rxcui, openfda_purpose
FROM drug_data
""", conn)
print(df.head())



     drug_name  rxcui openfda_purpose
0    Ibuprofen  12345     Pain relief
1     Warfarin  36870  Allergy relief
2  Simvastatin  47565  Blood thinning
3      Aspirin  10611     Pain relief
4   Tamsulosin  23867     Pain relief


In [14]:
#watching first 20 records
import sqlite3
import pandas as pd

conn = sqlite3.connect("medicine.db")

df = pd.read_sql_query("""
SELECT * FROM drug_data
LIMIT 20
""", conn)

print(df)



    id            drug_name  rxcui              openfda_warnings  \
0    1            Ibuprofen  12345              Use with caution   
1    3             Warfarin  36870                Take with food   
2    4          Simvastatin  47565                Take with food   
3    5              Aspirin  10611    Consult doctor if pregnant   
4    6           Tamsulosin  23867          May cause drowsiness   
5    7           Prednisone  97603              Use with caution   
6    8          Amoxicillin  52936    Consult doctor if pregnant   
7    9         Escitalopram  85693              Use with caution   
8   10           Duloxetine  35242              Use with caution   
9   11          Hydrocodone  42822    Consult doctor if pregnant   
10  12            Metformin  45309    Consult doctor if pregnant   
11  13         Escitalopram  15563                Take with food   
12  14        Ciprofloxacin  38633        Monitor blood pressure   
13  15           Sertraline  98800    Consult do

In [15]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("medicine.db")
df = pd.read_sql_query("SELECT * FROM drug_data", conn)

# Check missing values
missing_values = df.isnull().sum()
print("\nMissing Values per Column:\n", missing_values)

# Check duplicates based on the actual columns
duplicates = df.duplicated(subset=["drug_name", "rxcui", "openfda_warnings", "openfda_purpose"]).sum()
print(f"\nNumber of Duplicate Entries: {duplicates}")



Missing Values per Column:
 id                  0
drug_name           0
rxcui               0
openfda_warnings    0
openfda_purpose     0
dtype: int64

Number of Duplicate Entries: 1


In [16]:
# Remove duplicates in SQL
cursor = conn.cursor()

# Keep only the first occurrence of each duplicate row
cursor.execute("""
DELETE FROM drug_data
WHERE id NOT IN (
    SELECT MIN(id)
    FROM drug_data
    GROUP BY drug_name, rxcui, openfda_warnings, openfda_purpose
)
""")
conn.commit()
print("✅ Duplicate records removed")
conn.close()


✅ Duplicate records removed


In [18]:
# Uniqueness Check
duplicates = df.duplicated(subset=["drug_name", "rxcui", "openfda_warnings", "openfda_purpose"]).sum()
print(f"\nNumber of Duplicate Entries: {duplicates}")



Number of Duplicate Entries: 1
